In [13]:
from pathlib import Path

from bressen import BasisGegevens, Bressen, read_files

## Paden naar gegevens
Gegevens zijn aangeleverd in verschillende directories:
- 01_AGV_Dijktrajecten_Legger: dijktrajecten van waterschap Amstel Gooi en Vecht
- 02_ARK_Kering: kering van het Amsterdam-Rijnkanaal
- 03_Doorbraaklocaties: de locaties van de doorbraken
- 04_Watervlakken: alle beschikbare watervlakken

BASISGEGEVENS_GPKG en BRESSEN_GPKG verwijzen naar de basisgegevens (keringen, peil- en watervlakken) en het bressen-bestand

In [20]:
DATA_DIR = Path(r"d:\projecten\D2405.waternet.bressen\01.gegevens")
BASISGEGEVENS_GPKG = DATA_DIR / "basisgegevens.gpkg"
BRESSEN_GPKG = DATA_DIR / "bressen.gpkg"

## Inlezen
Inlezen van basisgegevens en bressen

In [21]:
# Inlezen basisgegevens vanuit een GeoPackage
basisgegevens = BasisGegevens.from_gpkg(BASISGEGEVENS_GPKG)

# Inlezen bressen vanuit meerdere bestanden naar een GeoDataFrame
bressen_gdf = read_files(
    [
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240122_breslocaties_gelijk_groter_1000.shp"),
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_kleiner_1000_angle.shp"),
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_Waterkeringen_Blaeu_ARK.shp"),
    ]
)

# Vullen van de bressenklasse met basisgegevens en de bressen GeoDataFrame
bressen = Bressen.from_gdf(bressen_gdf=bressen_gdf, basisgegevens=basisgegevens)

## Verwerken bressen
We hanteren de volgende instellingen:

- `offset`: offset van de bres t.o.v. kering in meter
- `breedte`: breedte van de bres haaks op de kering in meter
- `lengte`: lengte van de bres langs de kering in meter


### Complete set

In [16]:
bressen.bereken_vlakken(offset=50, lengte=200, breedte=40)

### Een of enkele bressen 
Wanneer je 1 of meerdere bressen afzonderlijk wilt processen met andere instellingen, kun je individuele bressen opvragen en als dusdanig processen.

In [17]:
bres = bressen.get_bres(fid=1329)
bres.bereken_vlak(offset=40, lengte=150, breedte=20, basisgegevens=bressen.basisgegevens)

## Bekijken resultaat
Resultaten kunnen worden opgevraagd als geodataframe. Hier een uitleg van de verschillend resultaat-velden:

### Gekozen instellingen
Velden met de gekozen instellingen voor het berekenen van het bresvlak:
* `bres_offset`: de gekozen offset t.o.v. de kering
* `bres_breedte`: de breedte van de bres haaks op de kering
* `bres_lengte`: de lengte van de bres in langsrichting van de kering

### Onderbouwing van het resultaat
Onder bouwing voor het wel, of niet, bepalen van het bresvlak:
* `bres_fout`: wanneer dit veld niet leeg is, staat er uitleg waarom het algoritme geen bres kan berekenen. Alle hieronder genoemde velden zijn dan leeg.
* `bres_offset_locatie`: de, door het algoritme, gekozen kant van de bres t.o.v. de kering. Wanneer er een watervlak naast de kering ligt, valt de keuze aan de ándere kant van de kering: `naast watervlak`. Zonder watervlak, maar met 2 peilgebieden met verschillende peilen, valt de keuze aan de kant van het `laagste peilgebied`. Is er maar aan 1 kant en peilgebied, dan valt de keuze aan de kant van het bestaande peilgebied, dus `binnen beheergebied`.


### Validatie van de resultaten
Als `True` in een van deze kolommen, dan is het goed deze bres te bekijken; deze is wellicht niet bruikbaar:
* `afwijking_oppervlak`: `True` wanneer `bres_lengte` * `bres_breedte` meer dan 2% afwijkt van het geometrisch oppervlak van het bresvlak
* `snijdt_kering`: `True` wanneer het berekende bresvlak een kering snijdt
* `snijdt_watervlak`: `True` wanneer het berekende bresvlak een watervlak snijdt

In [18]:
gdf = bressen.to_gdf(eigenschappen=False)

gdf

,bres_offset,bres_offset_locatie,bres_breedte,bres_lengte,bres_fout,geometry,afwijking_oppervlak,snijdt_kering,snijdt_watervlak
fid,,,,,,,,,
1,50,binnen beheergebied,40,200,None,"POLYGON ((129537.703 463435.395, 129539.941 46...",False,False,True
2,50,naast watervlak,40,200,None,"POLYGON ((129862.151 463153.456, 129862.272 46...",False,False,False
3,50,naast watervlak,40,200,None,"POLYGON ((130152.052 462768.863, 130152.345 46...",False,False,False
4,50,binnen beheergebied,40,200,None,"POLYGON ((130600.571 462599.507, 130601.947 46...",False,False,False
5,50,naast watervlak,40,200,None,"POLYGON ((130690.266 462167.927, 130690.223 46...",False,False,False
...,...,...,...,...,...,...,...,...,...
1545,50,naast watervlak,40,200,None,"POLYGON ((130039.882 477605.796, 130079.882 47...",False,False,True
1546,50,naast watervlak,40,200,None,"POLYGON ((130138.607 478106.108, 130178.607 47...",False,False,False
1547,50,naast watervlak,40,200,None,"POLYGON ((130136.903 478606.173, 130176.903 47...",False,False,False


## Wegschrijven resultaat

In de laatste stap schrijven we de resultaten weg naar een GeoPackage. Deze bevat drie lagen
- `bressen` met de originele punten samengevoegd in 1 laag (punt-feature)
- `bresvlakken` met de bresvlakken hierboven berekend (polygoon-feature)
- `fouten` met alle vlakken die niet kunnen worden berekend (geen geometrie)

In [19]:
bressen.to_gpkg(DATA_DIR / "resultaat.gpkg")